In [64]:
import numpy as np
import pandas as pd

import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

from sklearn.preprocessing import StandardScaler
import pickle


In [65]:
dataset = pd.read_csv(r"npk.csv")
dataset

,N,P,K,LABEL
0,65,50,50,Beans
1,75,60,80,Beans
2,88,70,60,Beans
3,99,80,50,Beans
4,100,55,70,Beans
...,...,...,...,...
65,102,44,79,watermelon
66,139,45,71,watermelon
67,129,48,75,watermelon
68,110,58,76,watermelon


In [66]:
labels = dataset['LABEL'].unique().tolist()
labels.sort()
labels

['Beans', 'okra', 'onion', 'potato', 'tomato', 'watermelon']

In [67]:
le = LabelEncoder()
le.fit(dataset['LABEL'])
dataset['LABEL']=le.transform(dataset['LABEL'])

In [68]:
dataset

,N,P,K,LABEL
0,65,50,50,0
1,75,60,80,0
2,88,70,60,0
3,99,80,50,0
4,100,55,70,0
...,...,...,...,...
65,102,44,79,5
66,139,45,71,5
67,129,48,75,5
68,110,58,76,5


In [50]:
scaler = StandardScaler()
#scaler.fit(X)
dataset.iloc[0:,:-1] =scaler.fit_transform(dataset.iloc[0:,:-1]) 

In [69]:

# selecting the features and labels
X = dataset.iloc[:, :-1].values
Y = dataset.loc[:, ['LABEL']]

print(X)
print(Y)

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# splitting the model into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=100)


[[ 65  50  50]
 [ 75  60  80]
 [ 88  70  60]
 [ 99  80  50]
 [100  55  70]
 [110  68  55]
 [120  78  66]
 [110  55  60]
 [103  50  65]
 [120  58  80]
 [134  60  75]
 [140  80  88]
 [145  75  60]
 [138  88  55]
 [150  60  50]
 [149  75  58]
 [119  87  60]
 [140  89  63]
 [122  66  56]
 [140  65  80]
 [156  77  60]
 [130  78  70]
 [160  80  64]
 [179  68  73]
 [208  78  80]
 [234  79  77]
 [256  67  57]
 [300  74  63]
 [239  70  73]
 [220  69  56]
 [213  68  67]
 [150  60  81]
 [113  65 112]
 [109  68 119]
 [149  80  85]
 [178  70  98]
 [189  78  93]
 [169  65 102]
 [120  79 111]
 [178  74 112]
 [199  88  98]
 [138  65  93]
 [160  67  90]
 [189  65 100]
 [167  78  92]
 [112  66  65]
 [107  65  78]
 [108  77  56]
 [150  66  89]
 [113  73  72]
 [109  62  73]
 [149  64  79]
 [178  78  73]
 [155  63  78]
 [189  76  80]
 [140  70  75]
 [189  62  78]
 [179  78  89]
 [156  71  50]
 [120  51  61]
 [140  60  69]
 [150  52  80]
 [130  56  77]
 [154  54  74]
 [123  60  63]
 [102  44  79]
 [139  45 

In [70]:

# # training a logistics regression model
# logmodel = LogisticRegression()
# logmodel.fit(X_train, y_train.values.ravel())
# predictions = logmodel.predict(X_test)
# print("***********************************************************")
# print("Accuracy = " + str(accuracy_score(y_test.values.ravel(), predictions)))
# print("***********************************************************")

sgd_classifier = SGDClassifier(tol=-np.infty, loss='log')
sgd_classifier.fit(X_train, y_train)

predictions = sgd_classifier.predict(X_test)
print("***********************************************************")
print("Accuracy = " + str(accuracy_score(y_test.values.ravel(), predictions)))
print("***********************************************************")

***********************************************************
Accuracy = 0.2777777777777778
***********************************************************


C:\Users\PRAMILA\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [71]:
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
 
# fit the regressor with x and y data
regressor.fit(X_train, y_train.values.ravel())
predictions = regressor.predict(X_train)
print("***********************************************************")

print(regressor.score(X_train, y_train))

print("***********************************************************")

***********************************************************
0.8696701832669322
***********************************************************


In [72]:
pickle.dump(regressor, open('randomforest_model.pkl','wb'))

In [73]:
test_data=[[0,0,144]]
print(regressor.predict([[0,42,144]]))

# Loading model to compare the results
svm_model = pickle.load(open('randomforest_model.pkl','rb'))
o=svm_model.predict(test_data)
print(o)
print(labels[int(o[0])])


[1.94]
[1.94]
okra


In [42]:
results = pd.DataFrame(sgd_classifier.predict_proba([[137,42,144]]).transpose())
results.rename(columns={0: "score"}, inplace=True)

results.reset_index(level=0, inplace=True)
results.sort_values(by="score", ascending=False, inplace=True, ignore_index=True)



In [43]:
for i in range(len(results)):
    if results.loc[i, "score"] > 0.001:
        print(str(labels[results.loc[i, "index"]]) + " (" + str(round(100 * results.loc[i, "score"])) + "%)")



onion (33%)
potato (33%)
watermelon (33%)
